<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/aya_expanse_8b/multigec/multigec_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!git clone https://$GIT_TOKEN@github.com/Reennon/multigec-models.git

Cloning into 'multigec-models'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 109 (delta 32), reused 68 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 125.11 KiB | 6.25 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [5]:
%cd multigec-models

/content/multigec-models


In [6]:
!git pull

Already up to date.


In [7]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade transformers trl -q
!pip install pandas numpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [60]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

from peft import PeftModel, PeftModelForCausalLM

tqdm.pandas()

In [10]:
parameters = OmegaConf.load("./params/aya_expanse_8b.yaml")

In [67]:
track     = "minimal"
fine_tuned_model_name = f"aya-expanse-8b-multigec"

hf_key   = userdata.get("hf_key")
secret_wandb = userdata.get("wandb_key")

in_path  = f"/content/drive/MyDrive/multigec/datasets/multigec_{track}.csv"
out_path = f"/content/drive/MyDrive/multigec/preds/multigec_test_{track}.csv"

out_model_dir = f"/content/drive/MyDrive/multigec/models/{fine_tuned_model_name}"
QUANTIZE_4BIT = True
device   = "cuda:0"

In [12]:
login(hf_key)

In [13]:
!env TORCH_USE_CUDA_DSA=1 -q

env: ‘-q’: No such file or directory


In [38]:
ls /content/drive/MyDrive/multigec/models

aya-expanse-8b-multigec/


In [62]:
base_model = "CohereForAI/aya-expanse-8b"
saved_checkpoint = out_model_dir + "/checkpoint-700"

quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(saved_checkpoint)
config = AutoConfig.from_pretrained(base_model)
base_model_instance = AutoModelForCausalLM.from_pretrained(
    base_model,
    config=config,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map=device,
    attn_implementation="flash_attention_2",
)
base_model_instance.resize_token_embeddings(len(tokenizer))
model = PeftModelForCausalLM.from_pretrained(
    base_model_instance,
    saved_checkpoint,
    torch_dtype=torch.bfloat16,
    device_map=device,
    ignore_mismatched_sizes=True
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:470: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([255040, 4096]) in the checkpoint and torch.Size([255029, 4096]) in the model instantiated
- base_model.model.model.embed_tokens.weight: found shape torch.Size([255040, 4096]) in the checkpoint and torch.Size([255029, 4096]) in the model instantiated.
  warnings.warn(msg)


In [68]:
multigec_df = pd.read_csv(in_path)
train_df = multigec_df.loc[multigec_df.loc[:, "split"] == "train"]
val_df = multigec_df.loc[multigec_df.loc[:, "split"] == "val"]
test_df = multigec_df.loc[multigec_df.loc[:, "split"] == "test"]

In [101]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>"

    return text

In [102]:
def make_correction(
    model,
    input: pd.Series,
    tokenizer,
    parameters: dict,
    seq_lenght: int
):
    input = formatting_prompts_func(input)
    input_ids = tokenizer(
        [input],
        padding=True,
        return_tensors="pt",
    ).input_ids
    input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])

    # Generate corrections
    gen_tokens = model.generate(
        input_ids,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=seq_lenght,
        do_sample=True,
    )
    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    correction = "".join(outputs)

    return correction

test_df[:5]["target"] = test_df[:5].progress_apply(lambda x: make_correction(
    model=model,
    input=x,
    tokenizer=tokenizer,
    parameters=parameters,
    seq_lenght=1600,
), axis=1)
test_df

100%|██████████| 5/5 [07:38<00:00, 91.65s/it]
<ipython-input-102-0620bec5f089>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[:5]["target"] = test_df[:5].progress_apply(lambda x: make_correction(


,language,essay_id,split,feature,target
2,czech,tei-skript2012_10-cb1ajanjak_01_1,test,Osnova: I. ÚVOD – Co chci\nII. STAŤ: a) černé ...,ebezpečí\nVyhrát!\nPrávě hraji nejdůležitější ...
8,czech,tei-skript2012_10-cb1akuband_02_1,test,Tak tady žiji ࿄ Z komínů stoupají dýmy spálené...,NaN
10,czech,tei-skript2012_10-cb1apexpet_01_1,test,: − pohled do zrcadla ࿄ −\n1) shrnutí obou obr...,NaN
14,czech,tei-skript2012_10-cb1ckaijan_02_1,test,Slohová práce školní\nJana Kaiserová\n17. květ...,NaN
17,czech,tei-skript2012_10-cb1cpolrob_02_1,test,Slohová práce školní\n17. května 2005\nTémata:...,NaN
...,...,...,...,...,...
23984,ukrainian,1364,test,"Будь-яка активність, яку можна об'ярликувати ц...",NaN
24219,ukrainian,1600,test,"""Як, - запитав я, - завадив вам?""\n\n- Своїм н...",NaN
24266,ukrainian,1647,test,"Якби не ревіння води, якби не розкати грому, я...",NaN
24332,ukrainian,1713,test,"Побачивши, як поводиться з вельми сумнівними в...",NaN


In [121]:
test_df[:1].feature.item()

Osnova: I. ÚVOD – Co chci
II. STAŤ: a) černé myšlenky
b) [1]„stav zóny“
c) rozhodující okamžik
III. ZÁVĚR – Co mi to dalo
ROZHODUJÍCÍ OKAMŽIK
Vyhrát!
Právě hraji nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.
Stav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“. Topíme se ve vlastní beznaději. Teď už chci jenom usnout a na všechno zapomenout. Nechci zase zažít tu ostudu, že jsme něco nedokázali.
Najednou vidím všechno jakoby jinýma očima. Dostal jsem asi nějaký impulz. Je to poslední záchvěv nebo nová síla? Je to zvláštní. Necítím únavu. Ta síla mne tlačí dál. A nejsem sá

In [120]:
print(test_df[:1].target.item())

ebezpečí
Vyhrát!
Právě hraji nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.
Stav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“. Topíme se ve vlastní beznaději. Teď už chci jenom usnout a na všechno zapomenout. Nechci zase zažít tu ostudu, že jsme něco nedokázali.
Najednou vidím všechno jakoby jinýma očima. Dostal jsem asi nějaký impulz. Je to poslední záchvěv nebo nová síla? Je to zvláštní. Necítím únavu. Ta síla mne tlačí dál. A nejsem sám! Najednou všichni jakoby ožívají. Týmu se zmocňuje tzv. Naganský duch. Nevymýšlím si. Nefantazíruji. Je to tak. Je to realita. Vidí

In [ ]:
raise Exception(e)

In [ ]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)
    target = example['target']

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

    return text

In [ ]:
num_added_toks = tokenizer.add_tokens(
    [v for v in LANG_CODE_TO_TOKEN.values()],
    special_tokens=True
)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
training_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

seq_length = 1300

cld_train_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=training_dataset,
    #dataset_text_field='feature',
    seq_length=seq_length,
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)
cld_val_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=val_dataset,
    #dataset_text_field='feature',
    seq_length=int(seq_length/2),
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)

In [ ]:
# parameters.training["gradient_accumulation_steps"] = 8
# parameters.training["per_device_train_batch_size"] = 5
# parameters.training["per_device_eval_batch_size"] = 2
# # Try these
# parameters.training["lr_scheduler_type"] = "cosine"
# parameters.training["max_grad_norm"] = 1.0
# parameters.training["output_dir"] = "results"
# parameters.training["num_train_epochs"] = 12
# parameters.training["eval_strategy"] = "steps"
# parameters.training["save_strategy"] = "steps"        # Save model after every step
# parameters.training["metric_for_best_model"] = "eval_loss"
# parameters.training["greater_is_better"] = False
# parameters.training["group_by_length"] = False # we already use CLD
# parameters.training["save_total_limit"] = 3           # Keep only the 3 most recent checkpoints
# parameters.training["load_best_model_at_end"] = True # EarlyStoppingCallback requires load_best_model_at_end = True
# parameters.training["warmup_steps"] = 50
# parameters.training["learning_rate"] = 3e-5
# parameters.training["save_steps"] = 25
# parameters.training["weight_decay"] = 0.0
# early_stopping_patience = 75

In [ ]:
cld_train_dataset

In [ ]:
dict(parameters.training)

In [ ]:
# def formatting_prompts_func_old(example):
#     output_texts = []
#     for i in range(len(example['inputs'])):
#         language_code = LANG_TO_CODE[example["language"]]
#         language_token = LANG_CODE_TO_TOKEN[language_code]

#         user_input = example['inputs'][i]
#         prompt_template = multigec_prompts[example["language"]]
#         instruction = prompt_template.format(original_text=user_input)
#         target = example['targets'][i]


#         text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"
#         output_texts.append(text)
#     return output_texts

# def formatting_prompts_func(example):
#     language_code = LANG_TO_CODE[example["language"]]
#     language_token = LANG_CODE_TO_TOKEN[language_code]

#     user_input = example['feature']
#     prompt_template = multigec_prompts[example["language"]]
#     instruction = prompt_template.format(original_text=user_input)
#     target = example['target']

#     text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

#     return text

In [ ]:
run = wandb.init(
    project=wandb_project_name,
    job_type="training",
    anonymous="allow"
)

wandb.config.update(dict(parameters.training))

peft_config = LoraConfig(
    r=parameters.lora.r,
    lora_alpha=parameters.lora.lora_alpha,
    target_modules=list(parameters.lora.target_modules),
    bias=parameters.lora.bias,
    task_type=parameters.lora.task_type
)
training_arguments = SFTConfig(
    **parameters.training,
    packing=True,
    max_seq_length=seq_length,
    output_dir=out_model_dir,
)
trainer = SFTTrainer(
    model=model,
    train_dataset=cld_train_dataset,
    eval_dataset=cld_val_dataset,
    peft_config=peft_config,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=parameters.early_stopping.early_stopping_patience)],
    #formatting_func=formatting_prompts_func,
)

with torch.backends.cuda.sdp_kernel(
    enable_flash=True,
    enable_math=False,
    enable_mem_efficient=False
):
    trainer.unload() # Should remove peft warnings
    trainer.train(resume_from_checkpoint=True)

In [ ]:
import time

run_name = wandb.run.name
model_out_path = f"/gdrive/MyDrive/models/multigec/{run_name}"
os.makedirs(model_out_path, exist_ok=True)
trainer.model.save_pretrained(model_out_path)
wandb.save(model_out_path + "/*")
model.config.use_cache = True
model.eval()
time.sleep(30)

In [ ]:
test_df = multigec_df.loc[multigec_df.loc[:, "split"] == "test"]

def make_correction(
    model,
    input: pd.Series,
    tokenizer,
    parameters: dict,
    seq_lenght: int
):
    language: str = input["language"]
    language_code: str = LANG_TO_CODE[language]
    language_token: str = LANG_CODE_TO_TOKEN[language_code]
    prompt_template: PromptTemplate = multigec_prompts[language]

    # Prepare inputs
    inputs: list[str] = sentences(input)
    inputs = [language_token + prompt_template.format(original_text=input) for input in inputs]
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])

    # Generate corrections
    gen_tokens = model.generate(
        input_ids,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=seq_lenght,
        do_sample=True,
    )
    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    correction = "".join(outputs)

    return correction

test_df.progress_apply(lambda x: make_correction(
    model=model,
    input=x,
    tokenizer=tokenizer,
    parameters=parameters,
    seq_lenght=seq_length,
))
test_df

In [ ]:
test_df.to_csv(out_path)

In [ ]:
wandb.finish()

In [ ]:
raise Exception e

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!pip install pynvml -q

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
gc.collect()

In [ ]:
import time
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

def wait_until_enough_gpu_memory(min_memory_available, max_retries=10, sleep_time=5):
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(torch.cuda.current_device())

    for _ in range(max_retries):
        clear_gpu_memory()
        info = nvmlDeviceGetMemoryInfo(handle)
        if info.free >= min_memory_available:
            break
        print(f"Waiting for {min_memory_available} bytes of free GPU memory. Retrying in {sleep_time} seconds...")
        time.sleep(sleep_time)
    else:
        raise RuntimeError(f"Failed to acquire {min_memory_available} bytes of free GPU memory after {max_retries} retries.")

# Usage example
min_memory_available = 38 * 1024 * 1024 * 1024  # 30GB
clear_gpu_memory()
wait_until_enough_gpu_memory(min_memory_available)

In [ ]:
del model

In [ ]:
del trainer